# Stress Analysis in 3d

In [6]:
h = 100.0  # height of the stand [mm]
M = 1.5  # the mass of the ball [kg]

E = 210.0  # Young's modulus
nu = 0.3  # Poisson's ratio

F = -10

In [7]:
import awkward as ak
import numpy as np
from polymesh.space import CartesianFrame
from polymesh.topo import detach_mesh_bulk
from sigmaepsilon.solid import Structure, FemMesh, PointData
from sigmaepsilon.solid.fem.cells import TET4 as CellData
from neumann.array import repeat, minmax

GlobalFrame = CartesianFrame(dim=3)

pddb = ak.from_parquet('stand_pointdata.parquet')
cddb = ak.from_parquet('stand_celldata.parquet')

coords = pddb['x'].to_numpy()
topo = cddb['nodes'].to_numpy().astype(int)
coords, topo = detach_mesh_bulk(coords, topo[:, :4])

A = np.array([
    [1, nu, nu, 0, 0, 0],
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0],
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E / (1-nu**2))

zmin, zmax = minmax(coords[:, 2])
h = zmax - zmin

cond = coords[:, 2] <= 0.005
ebcinds = np.where(cond)[0]
fixity = np.zeros((coords.shape[0], 6), dtype=bool)
fixity[ebcinds, :] = True

cond = coords[:, 2] >= 0.995 * h
nbcinds = np.where(cond)[0]
loads = np.zeros((coords.shape[0], 6))
loads[nbcinds, 2] = F/len(nbcinds)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = CellData(topo=topo, frames=frames)

# set up mesh and structure
mesh = FemMesh(pd, cd, model=A, frame=GlobalFrame)
structure = Structure(mesh=mesh)